In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install openpyxl

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [4]:
path_data = "/content/drive/MyDrive/Hybrid_DL_Kcr/data/"
path_model = "/content/drive/MyDrive/Hybrid_DL_Kcr/Model/"
path_result = "/content/drive/MyDrive/Hybrid_DL_Kcr/Model/"

In [5]:
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [6]:
# Train data
file_train = "train_data.csv"
df_train =pd.read_csv(path_data+file_train, delimiter= ',')

texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape

(13950, 2)

In [7]:
# load test data
file_test ="test_data.csv"
df_test =pd.read_csv(path_data+file_test,delimiter= ',')
text_test =[] #PTMsequend kmer
for i in df_test['Sequence']:
  temp = ProSentence(i,k)
  text_test.append(temp)
df_test['k_mer'] =text_test

test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)

Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest = np.array(df_test['Label'])
# perform one-hot encoding on the labels
ytest = np.array(ytest)
lb = LabelBinarizer()
ytest= lb.fit_transform(ytest)
ytest = to_categorical(ytest)
ytest.shape

(5978, 2)

In [8]:
max_sequence_length = MAX_SEQUENCE_LENGTH
INPUT_SIZE =300
vocab_size = 21

In [9]:
from tensorflow.keras import models, layers, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Embedding, Dropout, BatchNormalization, Dense

def create_model(vocab_size, max_sequence_length):
    inputs = Input(shape=(max_sequence_length,))
    embedding = Embedding(vocab_size + 1, 300, input_length=max_sequence_length, trainable=True)(inputs)

    # CNN with kernel size = 3
    cnn = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(embedding)
    cnn = MaxPooling1D(2)(cnn)
    #cnn = Dropout(0.2)(cnn)
    cnn = BatchNormalization()(cnn)

    # Fully Connected Layers
    flatten = Flatten()(cnn)
    flatten = Dropout(0.4)(flatten)
    dense1 = Dense(64, activation='relu')(flatten)
    outputs = Dense(2, activation='softmax')(dense1)  # Binary classification

    model = Model(inputs, outputs)
    return model



**Distillation học trên model teacher với bộ dl Generic**

In [10]:
from keras import models
from tensorflow.keras.optimizers import AdamW

model= create_model(vocab_size, MAX_SEQUENCE_LENGTH)
# Compile the model
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])
model.fit(Xtrain, ytrain,
              epochs=100,
              batch_size =16,
              )
model.save(path_model + "CNN_3.h5")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


872/872 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6261 - loss: 0.6856
Epoch 2/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7671 - loss: 0.4847
Epoch 3/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7765 - loss: 0.4587
Epoch 4/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7917 - loss: 0.4378
Epoch 5/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8039 - loss: 0.4202
Epoch 6/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8121 - loss: 0.4093
Epoch 7/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8188 - loss: 0.4016
Epoch 8/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8222 - loss: 0.3884
Epoch 9/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8299 - loss: 0.3703
Epoch 10/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8284 - loss: 0.3704
Epoch 11/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8446 - loss: 0.3507
Epoch 12/100
872/872 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step

In [11]:
import numpy as np
import json
import os  # Import os to check if the path exists
import pandas as pd  # Import pandas for easy CSV saving
from sklearn.metrics import confusion_matrix  # Import confusion_matrix


ypred = model.predict(Xtest)

# Ensure predictions are in the correct format
ypred = np.argmax(ypred, axis=1)  # Getting the predicted class
ytest_true = np.argmax(ytest, axis=1)  # Getting the true class

# Compute the confusion matrix
confusion_mat = confusion_matrix(ytest_true, ypred)  # Rename to avoid overwriting

# Calculate TP, TN, FP, FN
TN = confusion_mat[0, 0]  # True Negative
FP = confusion_mat[0, 1]  # False Positive
FN = confusion_mat[1, 0]  # False Negative
TP = confusion_mat[1, 1]  # True Positive

# Create a DataFrame to hold TP, TN, FP, FN values
data = {'TP': [TP],
        'FP': [FP],
        'TN': [TN],
        'FN': [FN]}

df = pd.DataFrame(data)

output_file = 'ID_CNN_3.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(os.path.join(path_result, output_file), index=False)

print(f"Các giá trị TP, TN, FP, FN đã được ghi vào file {output_file} dưới dạng cột.")


187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Các giá trị TP, TN, FP, FN đã được ghi vào file ID_CNN_3.xlsx dưới dạng cột.


In [12]:
#Cross-validation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
# Khởi tạo KFold với k=10
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# Khởi tạo các biến lưu trữ kết quả cross-validation
all_metrics = []
confusion_matrices = []

# Thực hiện K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")

    # Tạo dữ liệu train và validation cho fold hiện tại
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]

    model= create_model(vocab_size, MAX_SEQUENCE_LENGTH)
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train_fold, y_train_fold,
                  epochs=100,
                  batch_size =16,
                  )

    # Lưu lại kết quả cho từng fold
    results = model.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)

    # Dự đoán nhãn cho tập validation
    y_pred_fold = model.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # Tính confusion matrix và lưu TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Tính trung bình kết quả từ các fold
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(model.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Hiển thị các giá trị TP, TN, FP, FN cho từng fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# Hiển thị và lưu các giá trị TP, TN, FP, FN cho từng fold vào file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_CNN_3.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_CNN_3.xlsx'")

Training on fold 1/10...
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6252 - loss: 0.7044
Epoch 2/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7511 - loss: 0.5064
Epoch 3/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7770 - loss: 0.4683
Epoch 4/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8044 - loss: 0.4378
Epoch 5/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8061 - loss: 0.4221
Epoch 6/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8154 - loss: 0.4085
Epoch 7/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8136 - loss: 0.4042
Epoch 8/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8279 - loss: 0.3871
Epoch 9/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8217 - loss: 0.3876
Epoch 10/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8281 - loss: 0.3824
Epoch 11/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8321 - loss: 0.3756
Epoch 12/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6347 - loss: 0.6890
Epoch 2/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7420 - loss: 0.5145
Epoch 3/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7696 - loss: 0.4702
Epoch 4/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7880 - loss: 0.4422
Epoch 5/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8013 - loss: 0.4265
Epoch 6/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8064 - loss: 0.4129
Epoch 7/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8214 - loss: 0.3915
Epoch 8/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8302 - loss: 0.3820
Epoch 9/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8323 - loss: 0.3744
Epoch 10/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8434 - loss: 0.3552
Epoch 11/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8399 - loss: 0.3534
Epoch 12/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step

...

785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6349 - loss: 0.6726
Epoch 2/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7375 - loss: 0.5120
Epoch 3/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7813 - loss: 0.4660
Epoch 4/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7946 - loss: 0.4424
Epoch 5/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8059 - loss: 0.4201
Epoch 6/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8073 - loss: 0.4081
Epoch 7/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8235 - loss: 0.3899
Epoch 8/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8278 - loss: 0.3876
Epoch 9/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8337 - loss: 0.3761
Epoch 10/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8379 - loss: 0.3662
Epoch 11/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8517 - loss: 0.3477
Epoch 12/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6314 - loss: 0.6949
Epoch 2/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7466 - loss: 0.5117
Epoch 3/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7825 - loss: 0.4551
Epoch 4/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7992 - loss: 0.4356
Epoch 5/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8016 - loss: 0.4295
Epoch 6/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8183 - loss: 0.4073
Epoch 7/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8141 - loss: 0.4039
Epoch 8/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8298 - loss: 0.3796
Epoch 9/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8309 - loss: 0.3837
Epoch 10/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8383 - loss: 0.3613
Epoch 11/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8479 - loss: 0.3480
Epoch 12/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6273 - loss: 0.6818
Epoch 2/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7526 - loss: 0.5087
Epoch 3/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7838 - loss: 0.4560
Epoch 4/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7942 - loss: 0.4305
Epoch 5/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8011 - loss: 0.4275
Epoch 6/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8184 - loss: 0.4054
Epoch 7/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8185 - loss: 0.3984
Epoch 8/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8281 - loss: 0.3817
Epoch 9/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8285 - loss: 0.3775
Epoch 10/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8382 - loss: 0.3714
Epoch 11/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8523 - loss: 0.3426
Epoch 12/100
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step